<a href="https://colab.research.google.com/github/Nguyen379/Mask_detection/blob/main/bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pwd

'/content'

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/comap"

/content/drive/MyDrive/Colab Notebooks/comap


In [ ]:
# import packages
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams

rcParams['figure.figsize'] = 20, 10

from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('BCHAIN-MKPRU.csv')

df['Date'] = pd.to_datetime(df.Date, format="%m/%d/%y")
df.index = df['Date']

data = df.sort_index(ascending=True, axis=0)

new_data = pd.DataFrame(index=range(0, len(df)), columns=['Date', 'Value'])
print(data)


                 Date     Value
Date                           
2016-09-11 2016-09-11    621.65
2016-09-12 2016-09-12    609.67
2016-09-13 2016-09-13    610.92
2016-09-14 2016-09-14    608.82
2016-09-15 2016-09-15    610.38
...               ...       ...
2021-09-06 2021-09-06  51769.06
2021-09-07 2021-09-07  52677.40
2021-09-08 2021-09-08  46809.17
2021-09-09 2021-09-09  46078.38
2021-09-10 2021-09-10  46368.69

[1826 rows x 2 columns]


In [ ]:
for i in range(0, len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Value'][i] = data['Value'][i]

new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)
dataset = new_data.values
print(new_data.head())

             Value
Date              
2016-09-11  621.65
2016-09-12  609.67
2016-09-13  610.92
2016-09-14  608.82
2016-09-15  610.38


In [ ]:
eighty_percent = dataset.shape[0]*4//5
sixty_percent = dataset.shape[0]*3//5
print(sixty_percent)
train = dataset[sixty_percent:eighty_percent, :]
valid = dataset[eighty_percent:, :]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60, len(train)):
    x_train.append(scaled_data[i - 60:i, 0])
    y_train.append(scaled_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

In [ ]:
# predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)
X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i - 60:i, 0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)



In [ ]:
rms = np.sqrt(np.mean(np.power((valid - closing_price), 2)))
print(rms)

In [ ]:
train = new_data[sixty_percent:eighty_percent]
valid = new_data[eighty_percent:]
valid['Predictions'] = closing_price
plt.plot(train['Value'])
plt.plot(valid[['Value', 'Predictions']])